
## Upload YouTube Videos to the Reading Material Folder Manually


In this notebook, we will take URLs of videos, convert them to mp3s, upload them to Google Drive, trash them, and delete them.


We will start by ensuring that all the necessary python libraries are installed.

In [ ]:

import sys

print('conda update --all --yes --prefix "{}"'.format(sys.prefix))

In [ ]:

!{sys.executable} -m pip install --upgrade youtube_dl

In [ ]:

import os

# https://github.com/ytdl-org/youtube-dl/issues/20758
# Just replaced 'token' to 'account_playback_token' in line 1674 of extractor/youtube.py
# All seems to work.
for sub_dir, dirs_list, files_list in os.walk(r'C:\Users\dev\Anaconda3\Lib\site-packages\youtube_dl'):
    for file_name in files_list:
        if 'youtube.py' == file_name:
            print(os.path.join(sub_dir, file_name))

In [ ]:

!youtube-dl --version

In [ ]:

!{sys.executable} -m pip install --upgrade PyDrive


Next, we will prove to ourselves that we have ffmpeg correctly installed.

In [ ]:

# https://github.com/adaptlearning/adapt_authoring/wiki/Installing-FFmpeg
!ffmpeg -version


Performing authentication with Google Drive requires we keep our client id and consumer secret in a json file. Keep your secrets off the internet!

In [ ]:

!start %windir%\explorer.exe "C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\json"


We also make sure line 286 of pydrive's files.py file reads <code>self.content.close()</code> before we run the trash cell.

In [ ]:

file_path = 'C:\Users\dev\Anaconda3\Lib\site-packages\pydrive\files.py'
!"C:\Program Files\Notepad++\notepad++.exe" "{file_path}"


Before we start the download process, let's go over setting a few variables:

* **The downloads folder:** This is where all the messy work will be done. All mp3 files and any intermediate files will be deleted out of this folder by the end.
* **The URL list:** This is where we place our YouTube video URLs. It doesn't matter if you remove the <code>time_continue=</code> parameter in the URL, the mp3 will just start at zero seconds anyway.
* **The target folder ID:** This is the folder in Google Drive where our uploads will end up.

In [1]:

import youtube_dl

downloads_folder = r'C:\Users\577342\Downloads\\'
youtube_url_list = ['https://www.youtube.com/watch?v=4ODKBG5D4xI',
                    'https://www.youtube.com/watch?v=TMTrywlKnW4',
                    'https://www.youtube.com/watch?v=vgmt5JGVv3I',
                    'https://www.youtube.com/watch?v=laai-ve3KaI',
                    'https://www.youtube.com/watch?v=rgivGzjZixM',
                    'https://www.youtube.com/watch?v=CIP9mc35HQI',
                    'https://www.youtube.com/watch?v=rpndwf45nao',
                    'https://www.youtube.com/watch?v=FkRSrIfoGlg',
                    'https://www.youtube.com/watch?v=-RzA5aNZyIs',
                    'https://www.youtube.com/watch?v=nLFFNpTQla0',
                    'https://www.youtube.com/watch?v=3p2vobGtS0M']

Now we are ready to start the download process by running <code>youtube_dl</code> on our <code>youtube_url_list</code>.

In [2]:
ydl_opts = {
    'format': 'bestaudio/best',
    'nocheckcertificate': False,
    'outtmpl': downloads_folder+youtube_dl.DEFAULT_OUTTMPL,
    'postprocessors': [{'key': 'FFmpegExtractAudio',
                       'preferredcodec': 'mp3',
                       'preferredquality': '192'}],
    'verbose': True,
    }
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    try:
        result = ydl.download(youtube_url_list)
    except Exception as e:
        print(e)
print('Conversion completed.')

[debug] Encodings: locale cp1252, fs utf-8, out UTF-8, pref cp1252
[debug] youtube-dl version 2019.04.17
[debug] Python version 3.6.8 (CPython) - Windows-10-10.0.17763-SP0
[debug] exe versions: ffmpeg N-90173-gfa0c9d69d3, ffprobe N-90173-gfa0c9d69d3, phantomjs 2.1.1
[debug] Proxy map: {}


[youtube] 4ODKBG5D4xI: Downloading webpage
[youtube] 4ODKBG5D4xI: Downloading video info webpage
[debug] Invoking downloader on 'https://r1---sn-p5qlsnd6.googlevideo.com/videoplayback?id=o-AHVWNZz1BOW-zwkWzlYC5J0l7_i9X1_uUfccmuyOLkJY&itag=140&source=youtube&requiressl=yes&mm=31%2C29&mn=sn-p5qlsnd6%2Csn-p5qs7nes&ms=au%2Crdu&mv=m&pl=32&ei=A8O_XLyiDMvUwQHH9Jgg&gcr=us&initcwndbps=1977500&mime=audio%2Fmp4&gir=yes&clen=106429447&dur=6576.215&lmt=1555185092483501&mt=1556071046&fvip=1&keepalive=yes&c=WEB&txp=5431432&ip=2600%3A6c64%3A657f%3Af499%3A0%3Aab34%3A2e17%3A2a78&ipbits=0&expire=1556092771&sparams=ip%2Cipbits%2Cexpire%2Cid%2Citag%2Csource%2Crequiressl%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Cei%2Cgcr%2Cinitcwndbps%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&signature=2863A04C336BEAE164F3AA5C3F73174ED006F06A.9BA32C35EED544E492760DF7BBDBC765210E8A07&key=yt8&ratebypass=yes'
[download] Destination: C:\Users\577342\Downloads\Aion 22 ~ Carl Jung on The Archetype of Transformation-4ODKBG5D4xI.m4a
[download] 100% of 101.

[download] Destination: C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.m4a
[download] 100% of 85.49MiB in 00:08                          
[ffmpeg] Correcting container in "C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.m4a"
[debug] ffmpeg command line: ffmpeg -y -loglevel "repeat+info" -i "file:C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.m4a" -c copy -f mp4 "file:C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.temp.m4a"
[debug] ffmpeg command line: ffprobe -show_streams "file:C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.m4a"
[ffmpeg] Destination: C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.mp3
[debug] ffmpeg command line: ffmpeg -y -loglevel "repeat+info" -i "file:C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.m4a" -vn -acodec libmp3lame "-b:a" 192k "file:C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.mp3"
Deleting original file C:\Users\577342\Downloads\April 2019 Q and A-rgivGzjZixM.m4a (pass -k to

[debug] ffmpeg command line: ffmpeg -y -loglevel "repeat+info" -i "file:C:\Users\577342\Downloads\Aion 23 ~ Carl Jung on The Problem of Evil--RzA5aNZyIs.m4a" -vn -acodec libmp3lame "-b:a" 192k "file:C:\Users\577342\Downloads\Aion 23 ~ Carl Jung on The Problem of Evil--RzA5aNZyIs.mp3"
Deleting original file C:\Users\577342\Downloads\Aion 23 ~ Carl Jung on The Problem of Evil--RzA5aNZyIs.m4a (pass -k to keep)
[youtube] nLFFNpTQla0: Downloading webpage
[youtube] nLFFNpTQla0: Downloading video info webpage
[debug] Invoking downloader on 'https://r2---sn-p5qs7nel.googlevideo.com/videoplayback?id=o-AKajP-aqS-HyslOGnVvrcmVbSCepEHnyVvcYx_Q8Xmeh&itag=140&source=youtube&requiressl=yes&mm=31%2C29&mn=sn-p5qs7nel%2Csn-p5qlsnzd&ms=au%2Crdu&mv=m&pl=32&ei=sTPAXJCHK8KFwQG7jIbABg&initcwndbps=2883750&mime=audio%2Fmp4&gir=yes&clen=65368119&dur=4039.041&lmt=1555869947577588&mt=1556099923&fvip=2&keepalive=yes&c=WEB&txp=5531432&ip=2600%3A6c64%3A657f%3Af499%3A0%3Aab34%3A2e17%3A2a78&ipbits=0&expire=1556121617&


Next, we authenticate with Google by running the builtin web server conventiently provided.

In [3]:

from pydrive.auth import GoogleAuth

# The URL that gets displayed here is not suitable for public consumption
GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = r'../json/client_secrets.json'
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=756414784841-0nkbgkrice8gae3nc4ojafoukr8kb38e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.



Before you run the next cell, in the online web page of Google Drive, bring up the share dialog box of the folder you are uploading to and copy its folder id into the <code>tgt_folder_id</code> string. If you right-click on the folder and select **Share...** and then click **Get shareable link**, you will see something like:

https://drive.google.com/drive/folders/1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx?usp=sharing

It's that **1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx** that you need to copy.

In [4]:

from pydrive.drive import GoogleDrive

# Create GoogleDrive instance with authenticated GoogleAuth instance
drive = GoogleDrive(gauth)

# ID of the "Reading Material" folder
tgt_folder_id = '1syfUx6jukbW1CWIEy8xoM9veGGr5MBUh'


Next, we will upload all the mp3s in our download folder.

In [5]:

import os

# Upload all mp3s from the Downloads folder to Google Drive's "Reading Material" folder
gfile_dict = {}
for subdir, dirs, files in os.walk(downloads_folder):
    for src_file in files:
        if src_file.endswith('.mp3'):
            src_path = os.path.join(subdir, src_file)
            gfile_dict[src_file] = drive.CreateFile({'title':src_file, 'mimeType':'audio/mp3',
                                                     'parents': [{'kind': 'drive#fileLink',
                                                                  'id': tgt_folder_id}]})

            # Read mp3 file and set it as a content of this instance
            gfile_dict[src_file].SetContentFile(src_path)
            
            # Upload the file
            try:
                gfile_dict[src_file].Upload()
                print('Uploaded %s (%s)' % (gfile_dict[src_file]['title'],
                                            gfile_dict[src_file]['mimeType']))
            except Exception as e:
                print('Upload failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                         gfile_dict[src_file]['mimeType'], e))
print('Upload completed.')

Uploaded Aion 22 ~ Carl Jung on The Archetype of Transformation-4ODKBG5D4xI.mp3 (audio/mp3)
Uploaded Aion 23 ~ Carl Jung on The Problem of Evil--RzA5aNZyIs.mp3 (audio/mp3)
Uploaded April 2019 Q and A-rgivGzjZixM.mp3 (audio/mp3)
Uploaded Claire Lehmann & Quillette, Thinking Differently-TMTrywlKnW4.mp3 (audio/mp3)
Uploaded Did Nietzsche Hate Ugly People (Or Simply Love The Beautiful)-FkRSrIfoGlg.mp3 (audio/mp3)
Uploaded Ep. 14 - Awakening from the Meaning Crisis - Epicureans, Cynics, and Stoics-rpndwf45nao.mp3 (audio/mp3)
Uploaded Meme Analysis - What Are Memes-CIP9mc35HQI.mp3 (audio/mp3)
Uploaded Reality Check 2019, Jordan Greenhall-nLFFNpTQla0.mp3 (audio/mp3)
Uploaded The Cult of Logos is BRAINWASHING Young Men into Becoming Chad-laai-ve3KaI.mp3 (audio/mp3)
Uploaded The Heritage Foundation - Socialism & Personal Responsibility-3p2vobGtS0M.mp3 (audio/mp3)
Uploaded Understanding the Icons of Holy Week-vgmt5JGVv3I.mp3 (audio/mp3)
Upload completed.



Download the mp3s to <a href='http://www.voicedream.com/support/user-manual/#file'>Voice Dream</a> before you run the trash cell (below).

In [ ]:

import os

# Trash all mp3s from Google Drive's "Reading Material" folder
for src_file in gfile_dict.keys():

    # Trash mp3 file
    try:
        gfile_dict[src_file].Trash()
        print('Trashed %s (%s)' % (gfile_dict[src_file]['title'],
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Trash failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                gfile_dict[src_file]['mimeType'], e))
    
print('Trashing completed.')


Lastly, we run the code that will delete all the mp3s out of our downloads folder.

In [ ]:

import os

# Delete all mp3s in the Downloads folder
for src_file in gfile_dict.keys():
    src_path = os.path.join(downloads_folder, src_file)

    # Delete the file
    try:
        os.remove(src_path)
        print('Deleted %s (%s)' % (src_file,
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Failed to delete %s (%s): %s' % (src_file,
                                                gfile_dict[src_file]['mimeType'], e))
print('Deleting completed.')

In [ ]:

from IPython.display import HTML

rm_url = 'https://drive.google.com/drive/u/0/folders/{}'.format(tgt_folder_id)
message_str = 'open up your Google Drive target folder'
rm_link = '<a href="{}" target="_blank">{}</a>'.format(rm_url, message_str)
message_str = 'If you want to, you can {} and'.format(rm_link)
message_str += ' check if everything got deleted.'
HTML(message_str)

In [ ]:

!"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" {rm_url}